In [1]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Check your GPU name


True
NVIDIA GeForce GTX 1650 Ti


In [1]:
import os

model_path = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_summarization"  # Update this with the actual path
print(os.listdir(model_path))


['config.json', 'generation_config.json', 'model.safetensors', 'special_tokens_map.json', 'spiece.model', 'tokenizer_config.json']


In [2]:
import json

config_path = os.path.join(model_path, "config.json")
with open(config_path, "r") as f:
    config = json.load(f)

print(config.get("architectures", "Unknown Model"))


['PegasusForConditionalGeneration']


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

# Load the model and tokenizer from the local folder
model_path = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_summarization"  # Update this with the actual path
tokenizer = PegasusTokenizer.from_pretrained(model_path)
model = PegasusForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

def summarize_text(text):
    max_tokens = 1056
    input_tokens = tokenizer.encode(text, return_tensors="pt", truncation=False).shape[1]

    # Determine target summary length (25% of input length)
    target_length = max(1, int(input_tokens * 0.25))

    if input_tokens <= max_tokens:
        # Summarize normally
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_tokens).to(model.device)
        summary_ids = model.generate(**inputs, max_length=target_length, min_length=target_length//2, length_penalty=2.0, num_beams=4)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    else:
        # Chunking logic
        chunk_size = max_tokens - 56  # Keep some buffer space for special tokens
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
        summaries = []

        for chunk in chunks:
            inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=max_tokens).to(model.device)
            summary_ids = model.generate(**inputs, max_length=target_length//len(chunks), min_length=(target_length//len(chunks))//2, length_penalty=2.0, num_beams=4)
            summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

        return " ".join(summaries)  # Merge chunked summaries

# Example usage
text = """In Python, decorators are a powerful and flexible way to modify or extend the behavior of functions or methods, without changing their actual code. A decorator is essentially a function that takes another function as an argument and returns a new function with enhanced functionality.

Decorators are often used in scenarios such as logging, authentication and memorization, allowing us to add additional functionality to existing functions or methods in a clean, reusable way.Explanation of Parameters
1. decorator_name(func):

decorator_name: This is the name of the decorator function.
func: This parameter represents the function being decorated. When you use a decorator, the decorated function is passed to this parameter.
2. wrapper(*args, **kwargs):

wrapper: This is a nested function inside the decorator. It wraps the original function, adding additional functionality.
*args: This collects any positional arguments passed to the decorated function into a tuple.
**kwargs: This collects any keyword arguments passed to the decorated function into a dictionary.
The wrapper function allows the decorator to handle functions with any number and types of arguments.
3. @decorator_name:

This syntax applies the decorator to the function_to_decorate function. It is equivalent to writing function_to_decorate = decorator_name(function_to_decorate)."""
summary = summarize_text(text)
print(summary)


In [4]:
import os

model_path = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_summarization"
print(os.listdir(model_path))


['config.json', 'generation_config.json', 'model.safetensors', 'special_tokens_map.json', 'spiece.model', 'tokenizer_config.json']
